In [47]:
# !pip install -U torch transformers accelerate sentence-transformers
# !pip install -U torch transformers accelerate sentence-transformers
# !pip uninstall spellchecker -y
# !pip install pyspellchecker
# !pip install unidecode
# !pip install scikit-learn  # For cosine_similarity and numpy (it's a dependency)
# !pip install sentence-transformers
# !pip install pandas

import re
import unidecode
from spellchecker import SpellChecker
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

Leitura e padronização de colunas de chaves primarias para concatenação de dfs

In [54]:
pd.set_option('display.max_columns', None)
path = r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Transformed'

#Games 
df_list = []
for i in range(1, 8):
    df = pd.read_excel(path + f'\\0{i}_games.xlsx')
    df_list.append(df)

# db_id = 3  # 0 - 6
# display(df_list[db_id].describe())
# display(df_list[db_id].head(-3))
# display(df_list[db_id].info())

#fitragem série B comjumto 7
df_list[6] = df_list[6][df_list[6]['Série']=='A']

# mapeamento_colunas chave primária
mapeamento_colunas = {
    'data': 'Data',
    'Date': 'Data',
    'data_partida': 'Data',
    'time_mandante': 'Mandante',
    'mandante': 'Mandante',
    'Home': 'Mandante',
    'time_visitante': 'Visitante',
    'visitante': 'Visitante',
    'Away': 'Visitante',
}
df_list = [df.rename(columns=mapeamento_colunas) for df in df_list]

for i, df in enumerate(df_list):
    # # Para strings em formato brasileiro (DD/MM/AAAA)
    # df_list[i]['Data'] = pd.to_datetime(df_list[i]['Data'], format='%d/%m/%Y', errors='coerce')
    # # Mantenha apenas a data, removendo a parte do tempo
    # df_list[i]['Data'] = pd.to_datetime(df_list[i]['Data']).dt.date
    print(f"DataFrame {i}: Tipo da coluna 'Data' = {df['Data'].dtype}")
    print(f"Primeiros valores: {df['Data'].head(2).values}")
    print(f"Tem duplicatas? {df.duplicated(subset=['Data', 'Mandante', 'Visitante']).any()}")
    print("---")

DataFrame 0: Tipo da coluna 'Data' = datetime64[ns]
Primeiros valores: ['2003-03-29T00:00:00.000000000' '2003-03-29T00:00:00.000000000']
Tem duplicatas? False
---
DataFrame 1: Tipo da coluna 'Data' = object
Primeiros valores: ['29/03/2003' '29/03/2003']
Tem duplicatas? False
---
DataFrame 2: Tipo da coluna 'Data' = object
Primeiros valores: ['29/03/2003' '29/03/2003']
Tem duplicatas? False
---
DataFrame 3: Tipo da coluna 'Data' = object
Primeiros valores: ['29/3/2003' '29/3/2003']
Tem duplicatas? False
---
DataFrame 4: Tipo da coluna 'Data' = datetime64[ns]
Primeiros valores: ['2012-05-19T00:00:00.000000000' '2012-05-19T00:00:00.000000000']
Tem duplicatas? False
---
DataFrame 5: Tipo da coluna 'Data' = datetime64[ns]
Primeiros valores: ['2003-03-29T00:00:00.000000000' '2003-03-29T00:00:00.000000000']
Tem duplicatas? False
---
DataFrame 6: Tipo da coluna 'Data' = object
Primeiros valores: ['2012-05-20' '2012-05-20']
Tem duplicatas? False
---


In [57]:
def verificar_padrao_datas(df, coluna_data='Data'):
    """
    Verifica se todas as datas na coluna seguem o mesmo padrão.
    
    Args:
        df: DataFrame contendo a coluna de datas
        coluna_data: Nome da coluna que contém as datas
        
    Returns:
        dict: Dicionário com informações sobre os padrões encontrados
    """
    import re
    from collections import Counter
    
    # Pular se a coluna já for datetime
    if pd.api.types.is_datetime64_any_dtype(df[coluna_data]):
        return {
            'padrao_unico': True,
            'tipo': 'datetime64',
            'padroes': {'datetime64': len(df)}
        }
    
    # Função para detectar o padrão da data
    def detectar_padrao(data_str):
        if pd.isna(data_str):
            return 'NaN'
            
        # Remover espaços
        data_str = str(data_str).strip()
        
        # Padrões comuns
        padroes = {
            r'^\d{2}/\d{2}/\d{4}$': 'DD/MM/YYYY',
            r'^\d{1,2}/\d{1,2}/\d{4}$': 'D/M/YYYY',
            r'^\d{4}-\d{2}-\d{2}$': 'YYYY-MM-DD',
            r'^\d{2}-\d{2}-\d{4}$': 'DD-MM-YYYY',
            r'^\d{1,2}-\d{1,2}-\d{4}$': 'D-M-YYYY',
            r'^\d{2}\.\d{2}\.\d{4}$': 'DD.MM.YYYY',
            r'^\d{1,2}\.\d{1,2}\.\d{4}$': 'D.M.YYYY',
            r'^\d{4}/\d{2}/\d{2}$': 'YYYY/MM/DD'
        }
        
        for regex, nome_padrao in padroes.items():
            if re.match(regex, data_str):
                return nome_padrao
                
        return 'desconhecido'
    
    # Detectar o padrão de cada data
    padroes = df[coluna_data].apply(detectar_padrao)
    
    # Contar os padrões
    contagem_padroes = Counter(padroes)
    
    # Verificar se há apenas um padrão (excluindo NaN)
    padroes_sem_nan = {k: v for k, v in contagem_padroes.items() if k != 'NaN'}
    padrao_unico = len(padroes_sem_nan) == 1
    
    # Recuperar exemplos de cada padrão
    exemplos = {}
    for padrao in contagem_padroes.keys():
        if padrao != 'NaN':
            exemplos[padrao] = df.loc[padroes == padrao, coluna_data].iloc[0]
    
    return {
        'padrao_unico': padrao_unico,
        'padroes': dict(contagem_padroes),
        'exemplos': exemplos,
        'padrao_principal': padroes.value_counts().index[0] if not padroes.empty else None
    }


def padronizar_datas(df_list):
    """
    Padroniza todas as colunas de data nos DataFrames para datetime64[ns]
    
    Args:
        df_list: Lista de DataFrames com coluna 'Data'
        
    Returns:
        Lista de DataFrames com as datas padronizadas

    """

    import pandas as pd
    
    for i, df in enumerate(df_list):
        # Pular se já for datetime64
        if pd.api.types.is_datetime64_any_dtype(df['Data']):
            print(f"DataFrame {i}: Já é datetime64, mantido como está.")
            continue

        # Verificar o tipo de padrão predominante
        if all(df['Data'].str.contains('/', regex=False)):
            # Padrões DD/MM/YYYY ou D/M/YYYY (com separador '/')
            print(f"DataFrame {i}: Convertendo padrão com '/'")
            # Usar dayfirst=True para formatos DD/MM/YYYY e D/M/YYYY
            df['Data'] = pd.to_datetime(df['Data'], dayfirst=True, errors='coerce')
        
        elif all(df['Data'].str.contains('-', regex=False)):
            # Padrão YYYY-MM-DD (ISO)
            print(f"DataFrame {i}: Convertendo padrão ISO com '-'")
            df['Data'] = pd.to_datetime(df['Data'], errors='coerce')
            
        else:
            # Tentar formato misto para casos mais complexos
            print(f"DataFrame {i}: Tentando conversão com formato misto")
            df['Data'] = pd.to_datetime(df['Data'], format='mixed', dayfirst=True, errors='coerce')
        
        # Verificar se houve valores que não puderam ser convertidos
        num_nulos = df['Data'].isna().sum()
        if num_nulos > 0:
            print(f"  Atenção: {num_nulos} valores não puderam ser convertidos")
    
    return df_list

# # Exemplo de uso:
# for i, df in enumerate(df_list):
#     resultado = verificar_padrao_datas(df)
#     print(f"DataFrame {i}:")
#     print(f"  Padrão único: {resultado['padrao_unico']}")
#     print(f"  Padrões encontrados: {resultado['padroes']}")
#     if 'exemplos' in resultado:
#         print(f"  Exemplos: {resultado['exemplos']}")
#     #print(f"  Padrão principal: {resultado['padrao_principal']}")
#     print("---")

# Verificar os resultados
for i, df in enumerate(df_list):
    print(f"\nDataFrame {i}:")
    print(f"  Tipo da coluna 'Data': {df['Data'].dtype}")
    print(f"  Exemplo de valores: {df['Data'].head(2).values}")
    print(f"  Valores nulos: {df['Data'].isna().sum()}")


DataFrame 0:
  Tipo da coluna 'Data': datetime64[ns]
  Exemplo de valores: ['2003-03-29T00:00:00.000000000' '2003-03-29T00:00:00.000000000']
  Valores nulos: 0

DataFrame 1:
  Tipo da coluna 'Data': object
  Exemplo de valores: ['29/03/2003' '29/03/2003']
  Valores nulos: 0

DataFrame 2:
  Tipo da coluna 'Data': object
  Exemplo de valores: ['29/03/2003' '29/03/2003']
  Valores nulos: 0

DataFrame 3:
  Tipo da coluna 'Data': object
  Exemplo de valores: ['29/3/2003' '29/3/2003']
  Valores nulos: 0

DataFrame 4:
  Tipo da coluna 'Data': datetime64[ns]
  Exemplo de valores: ['2012-05-19T00:00:00.000000000' '2012-05-19T00:00:00.000000000']
  Valores nulos: 0

DataFrame 5:
  Tipo da coluna 'Data': datetime64[ns]
  Exemplo de valores: ['2003-03-29T00:00:00.000000000' '2003-03-29T00:00:00.000000000']
  Valores nulos: 0

DataFrame 6:
  Tipo da coluna 'Data': object
  Exemplo de valores: ['2012-05-20' '2012-05-20']
  Valores nulos: 0


In [46]:
for i, df in enumerate(df_list):
    # # Para strings em formato brasileiro (DD/MM/AAAA)
    # df_list[i]['Data'] = pd.to_datetime(df_list[i]['Data'], format='%d/%m/%Y', errors='coerce')
    # # Mantenha apenas a data, removendo a parte do tempo
    # df_list[i]['Data'] = pd.to_datetime(df_list[i]['Data']).dt.date
    print(f"DataFrame {i}: Tipo da coluna 'Data' = {df['Data'].dtype}")
    print(f"Primeiros valores: {df['Data'].head(2).values}")
    print(f"Tem duplicatas? {df.duplicated(subset=['Data', 'Mandante', 'Visitante']).any()}")
    print("---")

DataFrame 0: Tipo da coluna 'Data' = datetime64[ns]
Primeiros valores: ['2003-03-29T00:00:00.000000000' '2003-03-29T00:00:00.000000000']
Tem duplicatas? False
---
DataFrame 1: Tipo da coluna 'Data' = object
Primeiros valores: ['29/03/2003' '29/03/2003']
Tem duplicatas? False
---
DataFrame 2: Tipo da coluna 'Data' = object
Primeiros valores: ['29/03/2003' '29/03/2003']
Tem duplicatas? False
---
DataFrame 3: Tipo da coluna 'Data' = object
Primeiros valores: ['29/3/2003' '29/3/2003']
Tem duplicatas? False
---
DataFrame 4: Tipo da coluna 'Data' = datetime64[ns]
Primeiros valores: ['2012-05-19T00:00:00.000000000' '2012-05-19T00:00:00.000000000']
Tem duplicatas? False
---
DataFrame 5: Tipo da coluna 'Data' = datetime64[ns]
Primeiros valores: ['2003-03-29T00:00:00.000000000' '2003-03-29T00:00:00.000000000']
Tem duplicatas? False
---
DataFrame 6: Tipo da coluna 'Data' = object
Primeiros valores: ['2012-05-20' '2012-05-20']
Tem duplicatas? False
---


In [ ]:
mandante_list = []
visitante_list = []
for i in range(len(df_list)):
    mandante_list += df_list[i]['Mandante'].unique().tolist()
    visitante_list += df_list[i]['Visitante'].unique().tolist()

visitante_list_unique = list(set(visitante_list))
mandante_list_unique = list(set(mandante_list))
clubes_serie_a_2003_2025 = ["América Mineiro", "América Natal", "Athletico Paranaense", "Atlético Goianiense", "Atlético Mineiro", "Avaí", "Bahia", 
                            "Barueri", "Botafogo", "Red Bull Bragantino", "Brasiliense", "Ceará", 
                            "Chapecoense", "Corinthians", "Coritiba", "Criciúma", "Cruzeiro", "CSA", "Cuiabá", "Figueirense", "Flamengo", "Fluminense", 
                            "Fortaleza","Guarani", "Goiás", "Grêmio", "Grêmio Prudente", "Internacional", "Ipatinga", "Joinville", "Juventude","Mirassol", "Náutico", 
                            "Palmeiras", "Paraná", "Paysandu", "Ponte Preta", "Portuguesa", "Santa Cruz", "Santo André", "Santos", "São Caetano", "São Paulo", 
                            "Sport Recife", "Vasco da Gama", "Vitória"]


def corrigir_times_semanticamente(lista_times, lista_referencia, modelo=None):
    """
    Compara semanticamente os nomes de times com uma lista de referência.
    
    Parâmetros:
        lista_times (list): Lista com nomes possivelmente errados ou alternativos.
        lista_referencia (list): Lista com nomes corretos (padrão).
        modelo: Um modelo SentenceTransformer. Se None, será carregado 'all-MiniLM-L6-v2'.
    
    Retorna:
        List[Tuple[str, str, float]]: Lista com (nome_original, nome_corrigido, similaridade).
    """
    if modelo is None:
        modelo = SentenceTransformer('all-MiniLM-L6-v2')
    
    emb_ref = modelo.encode(lista_referencia)
    resultados = []

    for time in lista_times:
        emb_time = modelo.encode([time])
        sims = cosine_similarity(emb_time, emb_ref)[0]
        idx_mais_similar = np.argmax(sims)
        nome_corrigido = lista_referencia[idx_mais_similar]
        similaridade = sims[idx_mais_similar]
        resultados.append((time, nome_corrigido, round(similaridade, 1)))
    
    return resultados

def exibir_times_agrupados_por_referencia(resultados, lista_referencia, limite=0.5, limite_sup=1):
    """
    Exibe os times agrupados por nome de referência com suas similaridades.
    
    Parâmetros:
        resultados: Saída da função corrigir_times_semanticamente.
        lista_referencia: Lista com os nomes corretos.
        limite: Similaridade mínima para considerar um match (default=0.5).
    """
    from collections import defaultdict

    agrupados = defaultdict(list)

    # Agrupar por nome corrigido
    for original, corrigido, score in resultados:
        if score >= limite and score < limite_sup:
            agrupados[corrigido].append(f"{original} ({score})")

    # Exibir os nomes da lista de referência em ordem
    for nome in sorted(lista_referencia):
        similares = agrupados.get(nome, [])
        if similares:
            print(f"{nome}: {', '.join(similares)}")

# Criar dicionário a partir dos resultados da correção semântica
def gerar_dicionario_mapeamento(resultados, limite=0.5):
    """
    Gera um dicionário que mapeia nomes originais para nomes corrigidos, com base na similaridade.
    
    Parâmetros:
        resultados (list): Saída da função corrigir_times_semanticamente.
        limite (float): Similaridade mínima para considerar um match.
        
    Retorna:
        dict: Mapeamento {nome_original: nome_corrigido}
    """
    mapeamento = {
        original: corrigido 
        for original, corrigido, score in resultados 
        if score >= limite
    }
    return mapeamento

#resultados = corrigir_times_semanticamente(visitante, clubes_serie_a_2003_2025)

# for i, (original, corrigido, score) in enumerate(resultados):
#   if score < 1:
#     print(f"{i}:{original} → {corrigido} (similaridade: {score})")

# Mapeamento pois esses times não estão na lista de referencia e cruzando com alta similaridade incorreta
map_clubes = {
    "America-RN":"América Natal",
    "América-RN":"América Natal",
    "América rn":"América Natal",
    "Joinvile":"Joinville",
    "Atlético pr":"Athlético-PR",  
    "Atlético-PR":"Athlético-PR",
    "Grêmio Prudente":"Barueri",
    "Grêmio prudente":"Barueri",
    "Gremio Prudente":"Barueri"     
}

visitante_map,mandante_map = list(map(lambda x: map_clubes.get(x, x), visitante_list_unique)),list(map(lambda x: map_clubes.get(x, x), mandante_list_unique))

modelo = SentenceTransformer('all-MiniLM-L6-v2')

resultados = corrigir_times_semanticamente(visitante_map, clubes_serie_a_2003_2025, modelo)
visitante_map = gerar_dicionario_mapeamento(resultados, limite=0.0)
# print("Visitantes que não tem similaridade 100% com os clubes da série A de 2003 a 2025\n")
# exibir_times_agrupados_por_referencia(resultados, clubes_serie_a_2003_2025, limite=0.0)

resultados = corrigir_times_semanticamente(mandante_map, clubes_serie_a_2003_2025, modelo)
mandante_map = gerar_dicionario_mapeamento(resultados, limite=0.0)
# print("\nMandantes que não tem similaridade 100% com os clubes da série A de 2003 a 2025\n")
# exibir_times_agrupados_por_referencia(resultados, clubes_serie_a_2003_2025, limite=0.0)

for i in range(len(df_list)):
    df_list[i]['Mandante'] = df_list[i]['Mandante'].map(mandante_map)
    df_list[i]['Visitante'] = df_list[i]['Visitante'].map(visitante_map)


In [ ]:
# adicionar coluna para cada df identificando a origem do dataframe
for i in range(len(df_list)):
    df_list[i]['Origem'] = f'{i+1}'
    # db_id = 3  # 0 - 6
    #display(df_list[i].describe())
    #display(df_list[i].head(-3))
    #display(df_list[i].info())


DataFrame 0: Tipo da coluna 'Data' = object
Primeiros valores: [datetime.date(2003, 3, 29) datetime.date(2003, 3, 29)]
Tem duplicatas? False
---
DataFrame 1: Tipo da coluna 'Data' = object
Primeiros valores: [datetime.date(2003, 3, 29) datetime.date(2003, 3, 29)]
Tem duplicatas? False
---
DataFrame 2: Tipo da coluna 'Data' = object
Primeiros valores: [datetime.date(2003, 3, 29) datetime.date(2003, 3, 29)]
Tem duplicatas? False
---
DataFrame 3: Tipo da coluna 'Data' = object
Primeiros valores: [datetime.date(2003, 3, 29) datetime.date(2003, 3, 29)]
Tem duplicatas? False
---
DataFrame 4: Tipo da coluna 'Data' = object
Primeiros valores: [datetime.date(2012, 5, 19) datetime.date(2012, 5, 19)]
Tem duplicatas? False
---
DataFrame 5: Tipo da coluna 'Data' = object
Primeiros valores: [datetime.date(2003, 3, 29) datetime.date(2003, 3, 29)]
Tem duplicatas? True
---
DataFrame 6: Tipo da coluna 'Data' = object
Primeiros valores: [datetime.date(2012, 5, 19) datetime.date(2012, 5, 19)]
Tem duplicat

In [ ]:
df_merged = df_list[0]
for i, df_next in enumerate(df_list[1:], 1):
    # Verifica se há duplicatas antes do merge
    duplicates = df_next.duplicated(subset=['Data', 'Mandante', 'Visitante'], keep=False)
    if duplicates.any():
        print(f"DataFrame {i} tem {duplicates.sum()} linhas duplicadas!")
    
    # Merge com tratamento para validação 
    df_merged = pd.merge(
        df_merged,
        df_next,
        on=['Data', 'Mandante', 'Visitante'],
        how='outer',
        suffixes=(f'_{i-1}', f'_{i}')
        # Sem validação, pois pode haver duplicatas
    )

DataFrame 5 tem 1004 linhas duplicadas!


In [ ]:

# df_merged = df_list[0]
# for df_next in df_list[1:2]:
#     df_merged = pd.merge(
#         df_merged,
#         df_next,
#         on=['Data', 'Mandante', 'Visitante'],  # Suas três colunas chaves
#         how='outer',  # Ou outro tipo de join conforme necessidade
#         suffixes=(f'_{i}', f'_{i+1}')
#         #validate='one_to_one')

df_merged = pd.merge(df_list[0], df_list[1], on=['Data', 'Mandante', 'Visitante'], how='outer', suffixes=('_1', '_2'))
#df_merged.shape()
df_merged.describe()
df_merged.info()
df_merged.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11779 entries, 0 to 11778
Data columns (total 51 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Ano_1                           8453 non-null   float64       
 1   Data                            11779 non-null  datetime64[ns]
 2   rodada                          8453 non-null   float64       
 3   estadio                         8442 non-null   object        
 4   arbitro                         6743 non-null   object        
 5   publico                         6742 non-null   float64       
 6   publico_max                     4181 non-null   float64       
 7   Mandante                        11453 non-null  object        
 8   Visitante                       11448 non-null  object        
 9   tecnico_mandante_1              6299 non-null   object        
 10  tecnico_visitante_1             6299 non-null   object        
 11  co

(11779, 51)

In [ ]:
display(len(df_list[0:]))

7

In [ ]:
for i in range(len(df_list)):
    print(len(sorted(df_list[i]['Mandante'].dropna().unique())))
    print(len(sorted(df_list[i]['Visitante'].dropna().unique())))

    print(sorted(df_list[i]['Mandante'].dropna().unique()))
    print(sorted(df_list[i]['Visitante'].dropna().unique()))




43
43
['América Mineiro', 'Athletico Paranaense', 'Atlético Goianiense', 'Atlético Mineiro', 'Avaí', 'Bahia', 'Barueri', 'Botafogo', 'Brasiliense', 'CSA', 'Ceará', 'Chapecoense', 'Corinthians', 'Coritiba', 'Criciúma', 'Cruzeiro', 'Cuiabá', 'Figueirense', 'Flamengo', 'Fluminense', 'Fortaleza', 'Goiás', 'Grêmio', 'Guarani', 'Internacional', 'Ipatinga', 'Joinville', 'Juventude', 'Náutico', 'Palmeiras', 'Paraná', 'Paysandu', 'Ponte Preta', 'Portuguesa', 'Red Bull Bragantino', 'Santa Cruz', 'Santo André', 'Santos', 'Sport Recife', 'São Caetano', 'São Paulo', 'Vasco da Gama', 'Vitória']
['América Mineiro', 'Athletico Paranaense', 'Atlético Goianiense', 'Atlético Mineiro', 'Avaí', 'Bahia', 'Barueri', 'Botafogo', 'Brasiliense', 'CSA', 'Ceará', 'Chapecoense', 'Corinthians', 'Coritiba', 'Criciúma', 'Cruzeiro', 'Cuiabá', 'Figueirense', 'Flamengo', 'Fluminense', 'Fortaleza', 'Goiás', 'Grêmio', 'Guarani', 'Internacional', 'Ipatinga', 'Joinville', 'Juventude', 'Náutico', 'Palmeiras', 'Paraná', 'Pays

In [ ]:
# não esta validado

camp_2003 = ["América Mineiro", "Atlético Mineiro", "Atlético Paranaense", "Bahia", "Corinthians", "Coritiba", "Criciúma", "Cruzeiro", "Flamengo", "Fluminense", "Fortaleza", "Goiás", "Grêmio", "Guarani", "Internacional", "Juventude", "Palmeiras", "Paraná", "Ponte Preta", "Portuguesa", "Santos", "São Caetano", "São Paulo", "Vasco da Gama"]
camp_2004 = ["Atlético Mineiro", "Atlético Paranaense", "Botafogo", "Corinthians", "Coritiba", "Criciúma", "Cruzeiro", "Figueirense", "Flamengo", "Fluminense", "Goiás", "Grêmio", "Internacional", "Juventude", "Palmeiras", "Paraná", "Ponte Preta", "Santos", "São Caetano", "São Paulo", "Vasco da Gama", "Vitória"]
camp_2005 = ["Atlético Mineiro", "Atlético Paranaense", "Botafogo", "Brasiliense", "Corinthians", "Coritiba", "Cruzeiro", "Figueirense", "Flamengo", "Fluminense", "Fortaleza", "Goiás", "Internacional", "Juventude", "Palmeiras", "Paraná", "Ponte Preta", "Santos", "São Caetano", "São Paulo", "Vasco da Gama"]
camp_2006 = ["Atlético Paranaense", "Botafogo", "Corinthians", "Cruzeiro", "Figueirense", "Flamengo", "Fluminense", "Fortaleza", "Goiás", "Grêmio", "Internacional", "Juventude", "Palmeiras", "Paraná", "Ponte Preta", "Santa Cruz", "Santos", "São Caetano", "São Paulo", "Vasco da Gama"]
camp_2007 = ["América de Natal", "Atlético Mineiro", "Atlético Paranaense", "Botafogo", "Corinthians", "Cruzeiro", "Figueirense", "Flamengo", "Fluminense", "Goiás", "Grêmio", "Internacional", "Juventude", "Náutico", "Palmeiras", "Paraná", "Santos", "São Paulo", "Sport", "Vasco da Gama"]
camp_2008 = ["Atlético Mineiro", "Atlético Paranaense", "Botafogo", "Coritiba", "Cruzeiro", "Figueirense", "Flamengo", "Fluminense", "Goiás", "Grêmio", "Internacional", "Ipatinga", "Náutico", "Palmeiras", "Portuguesa", "Santos", "São Paulo", "Sport", "Vitória", "Vasco da Gama"]
camp_2009 = ["Atlético Mineiro", "Atlético Paranaense", "Avaí", "Barueri", "Botafogo", "Corinthians", "Coritiba", "Cruzeiro", "Flamengo", "Fluminense", "Goiás", "Grêmio", "Internacional", "Náutico", "Palmeiras", "Santo André", "Santos", "São Paulo", "Sport", "Vitória"]
camp_2010 = ["Atlético Goianiense", "Atlético Mineiro", "Atlético Paranaense", "Avaí", "Botafogo", "Ceará", "Corinthians", "Cruzeiro", "Flamengo", "Fluminense", "Goiás", "Grêmio", "Guarani", "Internacional", "Palmeiras", "Prudente", "Santos", "São Paulo", "Vitória", "Vasco da Gama"]
camp_2011 = ["América Mineiro", "Atlético Goianiense", "Atlético Mineiro", "Atlético Paranaense", "Avaí", "Bahia", "Botafogo", "Ceará", "Corinthians", "Coritiba", "Cruzeiro", "Figueirense", "Flamengo", "Fluminense", "Grêmio", "Internacional", "Palmeiras", "Santos", "São Paulo", "Vasco da Gama"]
camp_2012 = ["Atlético Goianiense", "Atlético Mineiro", "Bahia", "Botafogo", "Corinthians", "Coritiba", "Cruzeiro", "Figueirense", "Flamengo", "Fluminense", "Grêmio", "Internacional", "Náutico", "Palmeiras", "Ponte Preta", "Portuguesa", "Santos", "São Paulo", "Sport", "Vasco da Gama"]
camp_2013 = ["Atlético Mineiro", "Atlético Paranaense", "Bahia", "Botafogo", "Corinthians", "Coritiba", "Criciúma", "Cruzeiro", "Flamengo", "Fluminense", "Goiás", "Grêmio", "Internacional", "Náutico", "Palmeiras", "Ponte Preta", "Portuguesa", "Santos", "São Paulo", "Vasco da Gama"]
camp_2014 = ["Atlético Mineiro", "Atlético Paranaense", "Bahia", "Botafogo", "Chapecoense", "Corinthians", "Coritiba", "Criciúma", "Cruzeiro", "Figueirense", "Flamengo", "Fluminense", "Goiás", "Grêmio", "Internacional", "Palmeiras", "Santos", "São Paulo", "Sport", "Vitória"]
camp_2015 = ["Atlético Mineiro", "Atlético Paranaense", "Avaí", "Chapecoense", "Corinthians", "Coritiba", "Cruzeiro", "Figueirense", "Flamengo", "Fluminense", "Goiás", "Grêmio", "Internacional", "Joinville", "Palmeiras", "Ponte Preta", "Santos", "São Paulo", "Sport", "Vasco da Gama"]
camp_2016 = ["América Mineiro", "Atlético Mineiro", "Atlético Paranaense", "Botafogo", "Chapecoense", "Corinthians", "Coritiba", "Cruzeiro", "Figueirense", "Flamengo", "Fluminense", "Grêmio", "Internacional", "Palmeiras", "Ponte Preta", "Santa Cruz", "Santos", "São Paulo", "Sport", "Vitória"]
camp_2017 = ["Atlético Goianiense", "Atlético Mineiro", "Atlético Paranaense", "Avaí", "Bahia", "Botafogo", "Chapecoense", "Corinthians", "Coritiba", "Cruzeiro", "Flamengo", "Fluminense", "Grêmio", "Palmeiras", "Ponte Preta", "Santos", "São Paulo", "Sport", "Vasco da Gama", "Vitória"]
camp_2018 = ["América Mineiro", "Atlético Mineiro", "Atlético Paranaense", "Bahia","Botafogo", "Ceará", "Chapecoense", "Corinthians", "Cruzeiro", "Flamengo","Fluminense", "Grêmio", "Internacional", "Palmeiras", "Paraná","Santos", "São Paulo", "Sport", "Vasco da Gama", "Vitória"]
camp_2019 = ["Athletico Paranaense", "Atlético Mineiro", "Avaí", "Bahia", "Botafogo","Ceará", "Chapecoense", "Corinthians", "Cruzeiro", "CSA", "Flamengo","Fluminense", "Fortaleza", "Goiás", "Grêmio", "Internacional", "Palmeiras","Santos", "São Paulo", "Vasco da Gama"]
camp_2020 = ["Athletico Paranaense", "Atlético Goianiense", "Atlético Mineiro", "Bahia","Botafogo", "Ceará", "Corinthians", "Coritiba", "Flamengo", "Fluminense","Fortaleza", "Goiás", "Grêmio", "Internacional", "Palmeiras", "Bragantino","Santos", "São Paulo", "Sport", "Vasco da Gama"]
camp_2021 = ["América Mineiro", "Athletico Paranaense", "Atlético Goianiense","Atlético Mineiro", "Bahia", "Ceará", "Chapecoense", "Corinthians","Cuiabá", "Flamengo", "Fluminense", "Fortaleza", "Grêmio", "Internacional","Juventude", "Palmeiras", "Bragantino", "Santos", "São Paulo", "Sport" ]
camp_2022 = ["América Mineiro", "Athletico Paranaense", "Atlético Goianiense","Atlético Mineiro", "Avaí", "Botafogo", "Ceará", "Corinthians", "Coritiba","Cuiabá", "Flamengo", "Fluminense", "Fortaleza", "Goiás", "Internacional","Juventude", "Palmeiras", "Bragantino", "Santos", "São Paulo"]
camp_2023 = ["América Mineiro", "Athletico Paranaense", "Atlético Mineiro", "Bahia","Botafogo", "Corinthians", "Coritiba", "Cruzeiro", "Cuiabá", "Flamengo","Fluminense", "Fortaleza", "Goiás", "Grêmio", "Internacional", "Palmeiras","Bragantino", "Santos", "São Paulo", "Vasco da Gama"]
camp_2024 = ["Athletico Paranaense", "Atlético Mineiro", "Bahia", "Botafogo","Corinthians", "Criciúma", "Cruzeiro", "Cuiabá", "Flamengo", "Fluminense","Fortaleza", "Grêmio", "Internacional", "Juventude", "Palmeiras","Bragantino", "Santos", "São Paulo", "Vasco da Gama", "Vitória"]
camp_2025 = ["Atlético Mineiro", "Bahia", "Botafogo","Corinthians", "Criciúma", "Cruzeiro", "Cuiabá", "Flamengo", "Fluminense","Fortaleza", "Grêmio", "Internacional", "Juventude", "Palmeiras","Bragantino", "Santos", "São Paulo", "Vasco da Gama", "Vitória"]


# Criar uma lista única dos clubes de todos os campeonatos
clubes_serie_a = set(camp_2003 + camp_2004 + camp_2005 + camp_2006 + camp_2007 + camp_2008 + camp_2009 + camp_2010 + camp_2011 + camp_2012 + camp_2013 + camp_2014 + camp_2015 + camp_2016 + camp_2017 + camp_2018 + camp_2019 + camp_2020 + camp_2021 + camp_2022 + camp_2023 +camp_2024)

print(f"Total de clubes únicos: {len(clubes_serie_a)}")

Total de clubes únicos: 45
